In [20]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define cities
cities = [
    "Gampaha",
    "Belihuloya_Ratnapura",
    "Haputale_Badulla",
    "Kandy_City",
    "Matara_City"
]

# Load and compare data for each city
comparison_data = {}

for city in cities:
    # Load Solcast data
    solcast_file = f"solcast_nov_2025_{city}_hourly.csv"
    solcast_df = pd.read_csv(solcast_file)
    # Parse with timezone info, then convert to UTC
    solcast_df['period_end'] = pd.to_datetime(solcast_df['period_end'], utc=True)
    # Solcast period_end is at :30 minutes (end of period), subtract 30 min to align with OpenMeteo period start
    solcast_df['timestamp'] = solcast_df['period_end'] - pd.Timedelta(minutes=30)
    
    # Load OpenMeteo data
    openmeteo_file = f"openmeteo_{city}_nov_2025_hourly.csv"
    openmeteo_df = pd.read_csv(openmeteo_file)
    # OpenMeteo time has no timezone, so localize to Asia/Colombo first, THEN convert to UTC
    openmeteo_df['time'] = pd.to_datetime(openmeteo_df['time'])
    openmeteo_df['time'] = openmeteo_df['time'].dt.tz_localize('Asia/Colombo').dt.tz_convert('UTC')
    openmeteo_df['timestamp'] = openmeteo_df['time']  # OpenMeteo time is already period start
    
    # Merge data on aligned timestamp
    merged_df = pd.merge(
        solcast_df[['timestamp', 'ghi']],
        openmeteo_df[['timestamp', 'shortwave_radiation']],
        on='timestamp',
        how='outer'
    ).sort_values('timestamp')
    
    # Store for later use
    comparison_data[city] = merged_df
    
    print(f"{city}: {len(merged_df)} records loaded")

print("\nData loaded successfully for all cities!")

Gampaha: 726 records loaded
Belihuloya_Ratnapura: 726 records loaded
Haputale_Badulla: 726 records loaded
Kandy_City: 726 records loaded
Matara_City: 726 records loaded

Data loaded successfully for all cities!


In [22]:
# Create interactive comparison plots for each city
for city in cities:
    df = comparison_data[city]
    
    # Create figure
    fig = go.Figure()
    
    # Add Solcast GHI trace - now using aligned timestamp
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['ghi'],
        mode='lines',
        name='Solcast GHI',
        line=dict(color='blue', width=2),
        hovertemplate='<b>Solcast GHI</b><br>Time: %{x}<br>GHI: %{y} W/m²<extra></extra>'
    ))
    
    # Add OpenMeteo shortwave radiation trace - using same aligned timestamp
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['shortwave_radiation'],
        mode='lines',
        name='OpenMeteo Shortwave Radiation',
        line=dict(color='red', width=2),
        hovertemplate='<b>OpenMeteo</b><br>Time: %{x}<br>Radiation: %{y} W/m²<extra></extra>'
    ))
    
    # Update layout with zoom capabilities
    fig.update_layout(
        title=f'Solar Radiation Comparison - {city.replace("_", " ")}',
        xaxis_title='Date and Time',
        yaxis_title='Solar Radiation (W/m²)',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(
            rangeslider=dict(visible=True),
            type='date'
        )
    )
    
    fig.show()

In [23]:
# Statistical comparison for all cities
print("Statistical Comparison Summary\n" + "="*60)

for city in cities:
    df = comparison_data[city]
    
    # Calculate statistics
    ghi_mean = df['ghi'].mean()
    openmeteo_mean = df['shortwave_radiation'].mean()
    difference = ghi_mean - openmeteo_mean
    percent_diff = (difference / ghi_mean) * 100 if ghi_mean > 0 else 0
    
    print(f"\n{city.replace('_', ' ')}:")
    print(f"  Solcast GHI Mean:        {ghi_mean:.2f} W/m²")
    print(f"  OpenMeteo Mean:          {openmeteo_mean:.2f} W/m²")
    print(f"  Difference:              {difference:.2f} W/m² ({percent_diff:.2f}%)")

Statistical Comparison Summary

Gampaha:
  Solcast GHI Mean:        167.50 W/m²
  OpenMeteo Mean:          191.17 W/m²
  Difference:              -23.67 W/m² (-14.13%)

Belihuloya Ratnapura:
  Solcast GHI Mean:        152.76 W/m²
  OpenMeteo Mean:          181.08 W/m²
  Difference:              -28.32 W/m² (-18.54%)

Haputale Badulla:
  Solcast GHI Mean:        156.06 W/m²
  OpenMeteo Mean:          179.33 W/m²
  Difference:              -23.26 W/m² (-14.91%)

Kandy City:
  Solcast GHI Mean:        163.43 W/m²
  OpenMeteo Mean:          179.98 W/m²
  Difference:              -16.55 W/m² (-10.13%)

Matara City:
  Solcast GHI Mean:        191.92 W/m²
  OpenMeteo Mean:          198.05 W/m²
  Difference:              -6.14 W/m² (-3.20%)


In [21]:
# Debug: Check actual timestamps for Gampaha
print("Merged data - rows with OpenMeteo shortwave_radiation values:")
df_test = comparison_data['Gampaha']
print(df_test[df_test['shortwave_radiation'].notna()].head(15)[['timestamp', 'ghi', 'shortwave_radiation']])

print("\n" + "="*60)
print("\nMerged data - rows with both GHI and shortwave_radiation:")
print(df_test[(df_test['ghi'].notna()) & (df_test['shortwave_radiation'].notna())].head(15)[['timestamp', 'ghi', 'shortwave_radiation']])

Merged data - rows with OpenMeteo shortwave_radiation values:
                   timestamp    ghi  shortwave_radiation
0  2025-10-31 18:30:00+00:00    NaN                  0.0
1  2025-10-31 19:30:00+00:00    NaN                  0.0
2  2025-10-31 20:30:00+00:00    NaN                  0.0
3  2025-10-31 21:30:00+00:00    NaN                  0.0
4  2025-10-31 22:30:00+00:00    NaN                  0.0
5  2025-10-31 23:30:00+00:00    NaN                  0.0
6  2025-11-01 00:30:00+00:00   14.0                 15.0
7  2025-11-01 01:30:00+00:00  172.0                169.0
8  2025-11-01 02:30:00+00:00  405.0                422.0
9  2025-11-01 03:30:00+00:00  620.0                612.0
10 2025-11-01 04:30:00+00:00  726.0                774.0
11 2025-11-01 05:30:00+00:00  773.0                872.0
12 2025-11-01 06:30:00+00:00  856.0                893.0
13 2025-11-01 07:30:00+00:00  832.0                854.0
14 2025-11-01 08:30:00+00:00  744.0                721.0


Merged data - rows with 